# Hands-on disease module mining and drug prioritization

## Packages

In [1]:
import networkx as nx
import graph_tool as gt
import graph_tool.util as gtu
import graph_tool.topology as gtt
import graph_tool.centrality as gtc
import subprocess
import pandas as pd

## NetworkX vs graph-tool

### NetworkX
- https://networkx.org/
- **Pros:** widely used, tons of functions
- **Con:** mostly Python all the way down and thus slow

### graph-tool
- https://graph-tool.skewed.de/
- **Pro:** C++ backend (partly with OpenMP parallelization) and thus fast
- **Cons:** less widely used, fewer functions

**In this tutorial, we will show you how to use both of them**

## Task 1: Run disease module mining algorithms
-  Run the ROBUST algorithm with default parameters on the HD seeds and the PPI network provided in `data/NeDRex_api/`.
-  Use the `subprocess.run()` functions to run the command line interface.
-  Save the resulting disease module to `results/hd_module_robust.csv`.
-  Do the same for DIAMOnD and set the `n` parameter such that the size of the resulting disease module is identical to the disease module computed by ROBUST.
-  Save the resulting disease module to `results/hd_module_robust.csv`.

## Solutions to Task 1 

### Run ROBUST

In [2]:
result = subprocess.run(
    [
        'python3', 'robust.py', # Call to command line interface
        '../data/NeDRex_api/seed_genes_huntingtons_disease.csv', # Path to file with newline-separated seeds
        '../results/hd_module_robust.csv', # Path to output file
        '--network', '../data/NeDRex_api/filtered_ppi_only_reviewed_proteins_solution.csv', # Path to file with PPI network edge list
        '--namespace', 'UNIPROT' # Tell the tool that node IDs are UNIPROT IDs
    ],
    cwd='robust_bias_aware/'
)
print(f'stderr: {result.stderr}')
module_robust = pd.read_csv('results/hd_module_robust.csv')
print(f'Computed module with {len(module_robust[module_robust.terminal])} seeds and {len(module_robust[~module_robust.terminal])} non-seeds')

stderr: None
Computed module with 15 seeds and 60 non-seeds


### Run DIAMOnD (set-up to return module of equal size)

In [3]:
result = subprocess.run(
    [
        'python3', 'DIAMOnD.py', # Call to command line interface
        '../data/NeDRex_api/filtered_ppi_only_reviewed_proteins_solution.csv', # Path to file with PPI network edge list
        '../data/NeDRex_api/seed_genes_huntingtons_disease.csv', # Path to file with newline-separated seeds
        '58', # Number of non-seed nodes in disease module needs to be set explicitly
        '../results/hd_module_diamond.tsv' # Path to output file
    ],
    cwd='DIAMOnD/'
)
print(f'stderr: {result.stderr}')
module_diamond = pd.read_csv('results/hd_module_diamond.tsv', sep='\t')

/Users/dbblumenthal/teaching/courses/netmed-tutorial/DIAMOnD/DIAMOnD.py:180: SyntaxWarning: invalid escape sequence '\s'
  p-val = \sum_{n=kb}^{k} HypergemetricPDF(n,k,N,s)


DIAMOnD(): ignoring 2 of 17 seed genes that are not in the network

 results have been saved to '../results/hd_module_diamond.tsv' 

stderr: None


/Users/dbblumenthal/teaching/courses/netmed-tutorial/DIAMOnD/DIAMOnD.py:412: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p = float(DIAMOnD_node_info[3])


## Task 2: Inspect disaese modules with NetworkX and graph-tool
- Compute the following summary statistics for the disease modules computed by ROBUST and DIAMOnD:
     - Number of nodes.
     - Number of edges.
     - Density.
     - Number of connected components.
     - Average node degree.
     - Average node betweenness centrality.
- Provide both NetworkX and graph-tool implementations.
- Explain the observed differences between ROBUST and DIAMOnD disease modules.

## Solutions to Task 2

### Load PPI network with NetworkX

In [4]:
ppin_nx = nx.read_edgelist('data/NeDRex_api/filtered_ppi_only_reviewed_proteins_solution.csv', delimiter=',')
print(f'Loaded PPI network with {nx.number_of_nodes(ppin_nx)} nodes and {nx.number_of_edges(ppin_nx)} edges')

Loaded PPI network with 12797 nodes and 95916 edges


In [5]:
# Nodes are indexed with strings corresponding to IDs in input file
list(ppin_nx.nodes)[0]

'P15927'

### Load PPI network with graph-tool

In [6]:
ppin_gt = gt.load_graph_from_csv('data/NeDRex_api/filtered_ppi_only_reviewed_proteins_solution.csv')
print(f'Loaded PPI network with {ppin_gt.num_vertices()} nodes and {ppin_gt.num_edges()} edges')

Loaded PPI network with 12797 nodes and 95916 edges


In [7]:
# Nodes are indexed with integers from 0 to num_nodes - 1
# String node IDs from input file are stored in 'name' vertex property map
ppin_gt.vp['name'][ppin_gt.vertex(0)]

'P15927'

### Project DIAMOnD module on NetworkX and graph-tool graphs

#### Load seeds (not contained in output of DIAMOnD) and kick out seeds not in PPI network

In [8]:
with open('data/NeDRex_api/seed_genes_huntingtons_disease.csv') as fp:
    seeds = fp.read().splitlines()
seeds = [seed for seed in seeds if ppin_nx.has_node(seed)]

#### Project DIAMOnD and ROBUST modules on NetworkX graph

In [9]:
# DIAMOnD (seeds not contained in module file)
nx.set_node_attributes(ppin_nx, False, name='in_diamond_module')
for node in module_diamond.DIAMOnD_node:
    ppin_nx.nodes[node]['in_diamond_module'] = True
for seed in seeds:
    ppin_nx.nodes[seed]['in_diamond_module'] = True
module_diamond_nx = ppin_nx.subgraph([n for n, d in ppin_nx.nodes(data=True) if d['in_diamond_module']]).copy()

# ROBUST (seeds contained in module file)
nx.set_node_attributes(ppin_nx, False, name='in_robust_module')
for node in module_robust.vertex:
    ppin_nx.nodes[node]['in_robust_module'] = True
module_robust_nx = ppin_nx.subgraph([n for n, d in ppin_nx.nodes(data=True) if d['in_robust_module']]).copy()

#### Project DIAMOnD and ROBUST modules on graph-tool graph

In [10]:
# Lookup table to get node IDs from protein names
node_name_to_id = {ppin_gt.vp['name'][node]: node for node in ppin_gt.vertices()}

# DIAMOnD (seeds not contained in module file)
in_diamond_module = ppin_gt.new_vp('boolean', val=False)
for node in module_diamond.DIAMOnD_node:
    in_diamond_module[node_name_to_id[node]] = True
for seed in seeds:
    in_diamond_module[node_name_to_id[seed]] = True
ppin_gt.vp['in_diamond_module'] = in_diamond_module
module_diamond_gt = gt.GraphView(ppin_gt, vfilt=ppin_gt.vp['in_diamond_module'])

# ROBUST (seeds contained in module file)
in_robust_module = ppin_gt.new_vp('boolean', val=False)
for node in module_robust.vertex:
    in_robust_module[node_name_to_id[node]] = True
ppin_gt.vp['in_robust_module'] = in_robust_module
module_robust_gt = gt.GraphView(ppin_gt, vfilt=ppin_gt.vp['in_robust_module'])

### Functions to compute module statistics with NetworkX and graph-tool

In [27]:
def nx_summary_stats(module: nx.Graph):
    """
    Compute summary and average node-level statistics for a NetworkX subgraph (module).
    """
    n_nodes = module.number_of_nodes()
    n_edges = module.number_of_edges()
    density = nx.density(module)
    n_components = nx.number_connected_components(module)

    # Node-level averages
    avg_degree = sum(dict(module.degree()).values()) / n_nodes if n_nodes > 0 else 0
    betweenness = nx.betweenness_centrality(module) if n_nodes > 0 else {}
    avg_betweenness = sum(betweenness.values()) / n_nodes if n_nodes > 0 else 0

    return {
        "n_nodes": n_nodes,
        "n_edges": n_edges,
        "density": density,
        "n_components": n_components,
        "avg_degree": avg_degree,
        "avg_betweenness": avg_betweenness
    }

In [31]:
def gt_summary_stats(module):
    """
    Compute summary and average node-level statistics for a graph-tool subgraph (module).
    """
    n_nodes = module.num_vertices()
    n_edges = module.num_edges()
    density = (2 * n_edges) / (n_nodes * (n_nodes - 1)) if n_nodes > 1 else 0

    # Connected components
    comp, hist = gtt.label_components(module)
    n_components = len(hist)

    # Node-level averages
    degrees = [v.out_degree() for v in module.vertices()]
    avg_degree = sum(degrees) / n_nodes if n_nodes > 0 else 0

    if n_nodes > 0:
        vb, _ = gtc.betweenness(module)
        avg_betweenness = sum(vb[v] for v in module.vertices()) / n_nodes
    else:
        avg_betweenness = 0

    return {
        "n_nodes": n_nodes,
        "n_edges": n_edges,
        "density": density,
        "n_components": n_components,
        "avg_degree": avg_degree,
        "avg_betweenness": avg_betweenness,
    }

### Compute summary statistics for ROBUST and DIAMOnD modules with graph-tool and NetworkX

In [33]:
nx_summary_stats(module_robust_nx)

{'n_nodes': 73,
 'n_edges': 134,
 'density': 0.05098934550989345,
 'n_components': 1,
 'avg_degree': 3.671232876712329,
 'avg_betweenness': 0.03835187686239201}

In [29]:
nx_summary_stats(module_diamond_nx)

{'n_nodes': 73,
 'n_edges': 349,
 'density': 0.1328006088280061,
 'n_components': 11,
 'avg_degree': 9.561643835616438,
 'avg_betweenness': 0.0101989409822711}

In [34]:
gt_summary_stats(module_robust_gt)

{'n_nodes': 73,
 'n_edges': 134,
 'density': 0.05098934550989345,
 'n_components': 1,
 'avg_degree': 3.671232876712329,
 'avg_betweenness': 0.038351876862392005}

In [32]:
gt_summary_stats(module_diamond_gt)

{'n_nodes': 73,
 'n_edges': 349,
 'density': 0.1328006088280061,
 'n_components': 11,
 'avg_degree': 9.561643835616438,
 'avg_betweenness': 0.0101989409822711}

## Task 3: Prioritize drugs based on DIAMOnD and ROBUST modules

- Load the drug-protein interaction data from `data/NeDrex_api/pdi_solutions.csv` and map it to the PPI network.
- Ensure to add only those drug-protein interactions where the involved protein is contained in the PPI network.
- Run TrustRank (a.k.a. personalized PageRank) on the integrated protein-protein-drug interaction network to prioritize drugs targeting the ROBUST and DIAMOnD disease modules.
- Display the top 10 ranked drugs for both the ROBUST and the DIAMOnD module.
- Provide both NetworkX and graph-tool implementations. Measure runtimes of both implementations.

## Solution to Task 3

### Preparation: load PDI data into pandas data frame

In [58]:
pdi = pd.read_csv('data/NeDRex_api/pdi_solution.csv', header=None)
new_edges = [(pdi.loc[i,0],pdi.loc[i,1]) for i in range(pdi.shape[0]) if ppin_nx.has_node(pdi.loc[i,1])]
new_nodes = set(pdi.loc[i,0] for i in range(pdi.shape[0]) if ppin_nx.has_node(pdi.loc[i,1]))

### NetworkX implementation

#### Map drug-protein interactions on PPI network

In [60]:
ppi_pdi_nx = ppin_nx.copy()
nx.set_node_attributes(ppi_pdi_nx, 'protein', name='node_type')
ppi_pdi_nx.add_nodes_from(new_nodes, node_type='drug', in_robust_module=False, in_diamond_module=False)
ppi_pdi_nx.add_edges_from(new_edges)

#### Run personalized PageRank for ROBUST and DIAMOnD modules

In [142]:
%%timeit
personalization_robust = {n: 1.0 if d['in_robust_module'] else 0 for n, d in ppi_pdi_nx.nodes(data=True)}
pageranks_robust = nx.pagerank(ppi_pdi_nx, personalization=personalization_robust)
ranked_drugs_robust = sorted([(n, pageranks_robust[n]) for n, d in ppi_pdi_nx.nodes(data=True) if d['node_type'] == 'drug'], 
                             key = lambda t: t[1], reverse=True)

139 ms ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [143]:
%%timeit
personalization_diamond = {n: 1.0 if d['in_diamond_module'] else 0 for n, d in ppi_pdi_nx.nodes(data=True)}
pageranks_diamond = nx.pagerank(ppi_pdi_nx, personalization=personalization_diamond)
ranked_drugs_diamond = sorted([(n, pageranks_diamond[n]) for n, d in ppi_pdi_nx.nodes(data=True) if d['node_type'] == 'drug'], 
                             key = lambda t: t[1], reverse=True)

136 ms ± 799 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Display top 10 drugs

In [144]:
ranked_drugs_robust[:10]

[('DB04216', 0.0007292101736397119),
 ('DB00470', 0.000527720503436185),
 ('DB07159', 0.0004934666225662385),
 ('DB12010', 0.0004934666225662385),
 ('DB01268', 0.0004628558746283302),
 ('DB12500', 0.00046254921819304656),
 ('DB06595', 0.00043912009797590797),
 ('DB00675', 0.0004180963440651069),
 ('DB13245', 0.0004004847860757055),
 ('DB04224', 0.0003847886938398688)]

In [145]:
ranked_drugs_diamond[:10]

[('DB00331', 0.0011310517805650537),
 ('DB04216', 0.0007047786805121389),
 ('DB12141', 0.0005570557542185867),
 ('DB00470', 0.0005290614053794234),
 ('DB07159', 0.0004739416034476785),
 ('DB12010', 0.0004739416034476785),
 ('DB12500', 0.0004340665170150399),
 ('DB01268', 0.00043328147472263337),
 ('DB13245', 0.0004055422739900139),
 ('DB04224', 0.00039372029243624326)]

### graph-tool implementation

#### Map drug-protein interactions on PPI network

In [120]:
ppi_pdi_gt = ppin_gt.copy()
ppi_pdi_gt.vp['node_type'] = ppi_pdi_gt.new_vp('string', val='protein')

In [122]:
for drug in new_nodes:
    node = ppi_pdi_gt.add_vertex()
    ppi_pdi_gt.vp['name'][node] = drug
    ppi_pdi_gt.vp['node_type'][node] = 'drug'
    node_name_to_id[drug] = node
ppi_pdi_gt.add_edge_list([(node_name_to_id[p], node_name_to_id[d]) for p, d in new_edges])

#### Run personalized PageRank for ROBUST and DIAMOnD modules

In [146]:
%%timeit 
pagerank_robust_gt = gtc.pagerank(ppi_pdi_gt, pers=ppi_pdi_gt.vp['in_robust_module'])
ranked_drugs_robust_gt = sorted([(ppi_pdi_gt.vp['name'][i], pagerank_robust_gt[i]) for i in ppi_pdi_gt.vertices() if ppi_pdi_gt.vp['node_type'][i] == 'drug'], 
                                key = lambda t: t[1], reverse=True)

53.9 ms ± 1.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [147]:
%%timeit
pagerank_diamond_gt = gtc.pagerank(ppi_pdi_gt, pers=ppi_pdi_gt.vp['in_diamond_module'])
ranked_drugs_diamond_gt = sorted([(ppi_pdi_gt.vp['name'][i], pagerank_diamond_gt[i]) for i in ppi_pdi_gt.vertices() if ppi_pdi_gt.vp['node_type'][i] == 'drug'], 
                                key = lambda t: t[1], reverse=True)

52.3 ms ± 148 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Display top 10 drugs

In [135]:
ranked_drugs_robust_gt[:10]

[('DB04216', 0.05346093862679072),
 ('DB00470', 0.03937209370055734),
 ('DB07159', 0.035527038329590845),
 ('DB12010', 0.035527038329590845),
 ('DB12500', 0.03317763548862514),
 ('DB01268', 0.033166001428498564),
 ('DB06595', 0.03179446640897531),
 ('DB00675', 0.030303629626419228),
 ('DB13245', 0.029896766564815783),
 ('DB04224', 0.028695899906118857)]

In [138]:
ranked_drugs_diamond_gt[:10]

[('DB00331', 0.08336967207786936),
 ('DB04216', 0.050178410861116456),
 ('DB12141', 0.04030708076193223),
 ('DB00470', 0.03837486267674625),
 ('DB07159', 0.03333819605888466),
 ('DB12010', 0.03333819605888466),
 ('DB12500', 0.030400158305329543),
 ('DB01268', 0.030289629374345078),
 ('DB13245', 0.029481277659977264),
 ('DB04224', 0.028568623228189836)]